In [5]:
from math import degrees
from math import sqrt
from math import cos
from math import sin
from math import radians
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

def create_data_short():
    
    def headingFromCoordinates(lat1, lon1, lat2, lon2): 
        if(lon2-lon1 < 0): #2nd and 3rd quadrant 
            heading =180 - degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2)))) 
        elif(lat2-lat1 < 0): #4th quadrant
            heading = 360 + degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2))))
        else:
            heading = degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2)))) #1st quadrant            
        return heading
    df = pd.read_csv('Data/Raw_boat_data')
    df = df[210:6830]
    
    spd = []
    dgr = []
    
    for i in range(5):
        for i in range(47):
            spd.append(np.random.normal(0.5, 0.3))
            dgr.append(np.random.normal(107.5, 5))
        for i in range(9):
            spd.append(np.random.normal(0, 0.3))
            dgr.append(np.random.normal(7.5, 5))
        for i in range(47):
            spd.append(np.random.normal(-0.5, 0.3))
            dgr.append(np.random.normal(287.5, 5))
        for i in range(9):
            spd.append(np.random.normal(0, 0.3))
            dgr.append(np.random.normal(7.5, 5))
    
    
    for i in range(47):
        spd.append(np.random.normal(0.5, 0.3))
        dgr.append(np.random.normal(107.5, 5))
    for i in range(9):
        spd.append(np.random.normal(0, 0.3))
        dgr.append(np.random.normal(7.5, 5))
    for i in range(47):
        spd.append(np.random.normal(-0.5, 0.3))
        dgr.append(np.random.normal(287.5, 5))
    
    for i in range(len(dgr)):
        dgr[i] = dgr[i] + 140
        if dgr[i] > 360:
            dgr[i] = dgr[i] - 360    
    
    m = np.mean(df['acceleration'])
    s = np.std(df['acceleration'])
    
    acc = np.random.normal(m, s-0.2, 663)
    
    m0c = np.random.normal(0, 0.5, 663)
    # for i in range(5):
    #     m0c[47+(i*112)] = 10
    #     m0c[112+(i*112)] = -10

    m1c = np.random.normal(0, 0.5, 663)
    # for i in range(5):
    #     m1c[47+(i*112)] = -10
    #     m1c[112+(i*112)] = 10
        
    lats = [45.3596665575, 45.36225589, 45.36226747, 45.35967972, 45.35969351, 45.36227073,
            45.36228557, 45.35970006, 45.35971575, 45.36230304, 45.36230583,
            45.35971167, 45.35972416, 45.36232427, 45.36232928, 45.35974926,
            45.359755  , 45.36234141, 45.3623531 , 45.35975891, 45.35976923,
            45.36236122, 45.36236823, 45.3597792384]
    
    lons = [11.020597651, 11.01979384, 11.01985461, 11.02066511, 11.02074007, 11.01991931,
            11.0199881 , 11.02079678, 11.02087983, 11.02006662, 11.02012215,
            11.02093044 , 11.02100831, 11.02020208, 11.02025402, 11.02108572,
            11.02115422, 11.02033527, 11.02038858, 11.02118961, 11.02124338,
            11.02045991, 11.02053796, 11.0213068174]
    
    
    variance_lat = 0.000005
    variance_lon = 0.000002
    
    
    def noise_lon(coord):
        noise = coord - np.random.normal(coord, variance_lon)
        return coord + noise
    
    
    def noise_lat(coord):
        noise = coord - np.random.normal(coord, variance_lat)
        return coord + noise
    
    
    for i, elem in enumerate(lats):
        lats[i] = noise_lat(elem)   
    
    for j,elem in enumerate(lons):
        lons[j] = noise_lon(elem)
    
    
    g_lat = []
    
    g_lon = []
    
    for i in range(len(lats) - 1):
        start_lat = lats[i]
        start_lon = lons[i]
        g_lat.append(start_lat)
        g_lon.append(start_lon)
        
        l = sqrt((lons[i + 1] - lons[i]) ** 2 + (lats[i + 1] - lats[i]) ** 2)
        
        if i % 4 == 0:
            dl = l / 47
            alpha = 180 - headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, 47):
                g_lon.append(noise_lon((start_lon - (j * dl * cos(radians(alpha))))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
        if i % 4 == 1:
            dl = l / 9
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, 9):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
        if i % 4 == 2:
            dl = l / 47
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, 47):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
        if i % 4 == 3:
            dl = l / 9
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, 9):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
    
    sins = []
    cosins = []
    id_list = []
    for i, degree in enumerate(dgr):
        sins.append(np.sin(np.deg2rad(degree)))
        cosins.append(np.cos(np.deg2rad(degree)))
        
        
    columns = ["Speed", "Sin", "Cosin", 
               "Acceleration", "M0C", 
               "M1C", "Lat", "Lon"]
    
    # print(len(sins), len(cosins))
    # plt.plot(sins)
    # plt.plot(cosins)
    # plt.show()
    # 
    # print(len(dgr))
    # plt.plot(dgr)
    # plt.show()
    
    df = pd.DataFrame(columns=columns)
    df["Speed"] = spd
    df["Sin"] = sins
    df["Cosin"] = cosins
    df["Acceleration"] = acc
    df["M0C"] = m0c
    df["M1C"] = m1c
    df["Lat"] = g_lat
    df["Lon"] = g_lon
    df = df.drop(df.index[-7:])
    
    #df = df.drop(df.index[-39:])
    
 
    return df


In [4]:
def create_data():
    def headingFromCoordinates(lat1, lon1, lat2, lon2): 
        if(lon2-lon1 < 0): #2nd and 3rd quadrant 
            return 180 - degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2))))
        if(lat2-lat1 < 0): #4th quadrant
            return 360 + degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2))))
        return degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2)))) #1st quadrant
    
    df = pd.read_csv('Data/Raw_boat_data')
    df = df[210:6830]
    
    
    spd = []
    dgr = []
    
    for i in range(5):
        for i in range(470):
            spd.append(np.random.normal(0.5, 0.3))
            dgr.append(np.random.normal(107.5, 5))
        for i in range(9):
            spd.append(np.random.normal(0, 0.3))
            dgr.append(np.random.normal(7.5, 5))
        for i in range(470):
            spd.append(np.random.normal(-0.5, 0.3))
            dgr.append(np.random.normal(287.5, 5))
        for i in range(9):
            spd.append(np.random.normal(0, 0.3))
            dgr.append(np.random.normal(7.5, 5))
    
    #6
    for i in range(470):
        spd.append(np.random.normal(0.5, 0.3))
        dgr.append(np.random.normal(107.5, 5))
    for i in range(9):
        spd.append(np.random.normal(0, 0.3))
        dgr.append(np.random.normal(7.5, 5))
    for i in range(470):
        spd.append(np.random.normal(-0.5, 0.3))
        dgr.append(np.random.normal(287.5, 5))
    
    for i in range(len(dgr)):
        dgr[i] = dgr[i] + 140
        if(dgr[i] > 360):
            dgr[i] = dgr[i] -360   
    
    m = np.mean(df['acceleration'])
    s = np.std(df['acceleration'])
    
    acc = np.random.normal(m, s-0.2, 5739)
    
    m0c = np.random.normal(0, 0.5, 5739)

        
    m1c = np.random.normal(0, 0.5, 5739)

    
    lats = [45.3596665575, 45.36225589, 45.36226747, 45.35967972, 45.35969351, 45.36227073,
            45.36228557, 45.35970006, 45.35971575, 45.36230304, 45.36230583,
            45.35971167, 45.35972416, 45.36232427, 45.36232928, 45.35974926,
            45.359755  , 45.36234141, 45.3623531 , 45.35975891, 45.35976923,
            45.36236122, 45.36236823, 45.3597792384]
    
    lons = [11.020597651, 11.01979384, 11.01985461, 11.02066511, 11.02074007, 11.01991931,
            11.0199881 , 11.02079678, 11.02087983, 11.02006662, 11.02012215,
            11.02093044 , 11.02100831, 11.02020208, 11.02025402, 11.02108572,
            11.02115422, 11.02033527, 11.02038858, 11.02118961, 11.02124338,
            11.02045991, 11.02053796, 11.0213068174]
    
    variance_lat = 0.000005
    variance_lon = 0.000002
    
    def noise_lon(coord):
        noise = coord - np.random.normal(coord, variance_lon)
        return coord + noise
    
    
    def noise_lat(coord):
        noise = coord - np.random.normal(coord, variance_lat)
        return coord + noise
    
    
    for i, elem in enumerate(lats):
        lats[i] = noise_lat(elem)   
    
    for j,elem in enumerate(lons):
        lons[j] = noise_lon(elem)
    
    
    g_lat = []
    g_lon = []
    
    for i in range(len(lats) - 1):
        start_lat = lats[i]
        start_lon = lons[i]
        g_lat.append(start_lat)
        g_lon.append(start_lon)
        
        l = sqrt((lons[i + 1] - lons[i]) ** 2 + (lats[i + 1] - lats[i]) ** 2)
        
        if i % 4 == 0:
            dl = l / 470
            alpha = 180 - headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, 470):
                g_lon.append(noise_lon((start_lon - (j * dl * cos(radians(alpha))))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
        if i % 4 == 1:
            dl = l / 9
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, 9):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
        if i % 4 == 2:
            dl = l / 470
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, 470):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
        if i % 4 == 3:
            dl = l / 9
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, 9):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
    
                 
    columns = ["Speed", "Degrees", 
               "Acceleration", "M0C", 
               "M1C", "Lat", "Lon"]
    
 
    df = pd.DataFrame(columns=columns)
    df["Speed"] = spd
    df["Degrees"] = dgr
    df["Acceleration"] = acc
    df["M0C"] = m0c
    df["M1C"] = m1c
    df["Lat"] = g_lat
    df["Lon"] = g_lon
    
    plt.plot(df["Lon"], df["Lat"])
    plt.show()
    
    return df



In [8]:
df = create_data_short()
for i in range(656):
    data = create_data_short()
    # choice = np.random.choice(3, 1)
    # if choice == 0:
    #     data[331:] = np.zeros((663-331,7))
    # elif choice == 1:
    #     data[:332] = np.zeros((332,7))
    df = df.append(data)
    
print(df)

        Speed       Sin     Cosin  Acceleration       M0C       M1C  \
0    0.684053 -0.905635 -0.424059     -0.020241  0.760359  0.839897   
1    0.532341 -0.930357 -0.366654     -0.494499  0.509074  0.025151   
2    0.025358 -0.886622 -0.462495     -0.437766  0.575773 -0.116081   
3    0.849382 -0.940413 -0.340034      0.270971 -0.362658  0.004421   
4    0.465634 -0.913886 -0.405970     -0.987736 -0.323611 -0.475731   
..        ...       ...       ...           ...       ...       ...   
651 -0.581838  0.895722  0.444614     -0.336114  0.550401  0.217290   
652 -1.044861  0.897496  0.441022     -0.147302 -0.117672 -0.497628   
653 -0.400650  0.959958  0.280145     -1.308335  0.089513 -0.097186   
654 -0.065780  0.917120  0.398612     -0.400937  0.623984  0.132699   
655 -0.339843  0.945779  0.324810      1.044522  0.229273 -0.256973   

           Lat        Lon  
0    45.359669  11.020597  
1    45.359726  11.020581  
2    45.359771  11.020564  
3    45.359829  11.020548  
4    45

In [6]:
df = create_data()
for i in range(10):
    df = df.append(create_data())
        
print(df)




         Speed     Degrees  Acceleration       M0C       M1C        Lat  \
0     0.550676  238.698506      0.473859  0.390617  0.173035  45.359664   
1     0.589577  248.627856     -0.394985 -0.195031 -0.147516  45.359674   
2     0.695733  245.456166     -0.428557 -0.952931 -0.008597  45.359678   
3     0.315313  249.014510      1.188197  0.365878 -0.554043  45.359679   
4     0.858325  237.032291      0.126009 -0.891212 -0.146225  45.359690   
...        ...         ...           ...       ...       ...        ...   
5734 -0.398674   72.720635     -0.722588  0.233087 -1.450635  45.359801   
5735 -0.557738   73.250594      0.626376 -1.003928  0.379707  45.359800   
5736 -0.649053   73.625468      0.238924 -0.198920  0.176160  45.359795   
5737 -0.985046   71.255775     -0.004200 -0.499482 -0.410731  45.359782   
5738 -0.183512   69.221523     -0.458903 -0.585247 -0.345293  45.359777   

            Lon  
0     11.020597  
1     11.020597  
2     11.020595  
3     11.020589  
4     11.

In [9]:
# for i, elem in enumerate(df_list): 
#     name = "Data/Boat_nominal_data/sequences/Boat_sequence^" + str(i) + ".csv"
#     elem.to_csv(name)

df = df.to_csv("Data/Boat_nominal_data/Boat_nominal_long.csv")
